In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import geopandas as gpd
import folium
from branca.element import MacroElement
from jinja2 import Template
from IPython.display import IFrame
import webbrowser

from libpysal.weights import Queen
from esda.moran import Moran

from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance

# Charger les données sauvegardées
gdf_factors = gpd.read_file("gdf_factors.geojson")


In [ ]:
gdf_factors.head()

In [ ]:
# Étape 1 : Nettoyage des géométries
gdf_factors = gdf_factors[gdf_factors.geometry.notnull()]
gdf_factors = gdf_factors[gdf_factors.is_valid]

# Étape 2 : Supprimer les géométries de type Point ou LineString si présentes
gdf_factors = gdf_factors[gdf_factors.geometry.geom_type.isin(['Polygon', 'MultiPolygon'])]

# Étape 3 : Sélection des colonnes numériques avec variance non nulle
numeric_columns = gdf_factors.select_dtypes(include='number').columns
numeric_columns = [col for col in numeric_columns if gdf_factors[col].std() > 0]

# Étape 4 : Création de la matrice de voisinage spatiale
w = Queen.from_dataframe(gdf_factors, use_index=True)
w.transform = 'r'

# Étape 5 : Calcul de Moran's I pour chaque facteur
results = []
for col in numeric_columns:
    try:
        moran = Moran(gdf_factors[col], w)
        results.append({
            'facteur': col,
            "Moran's I": round(moran.I, 4),
            'p-value': round(moran.p_sim, 4)
        })
    except Exception as e:
        results.append({
            'facteur': col,
            "Moran's I": None,
            'p-value': None,
            'erreur': str(e)
        })

# Résultats
moran_df = pd.DataFrame(results)
print(moran_df)


In [ ]:
# Diagramme des indices de Moran
plt.figure(figsize=(10, 6))
sns.barplot(data=moran_df.sort_values("Moran's I", ascending=False), x="Moran's I", y="facteur", palette="viridis")
plt.title("Indice de Moran par facteur")
plt.xlabel("Moran's I")
plt.ylabel("Facteur")
plt.tight_layout()
plt.savefig("moran_barplot.png")
plt.show()

In [ ]:
# Regression + Random Forest

In [ ]:


facteurs = [
    'demographie_niveau_vie', 'emploi_activite', 'acces_sante',
    'services_publics', 'agriculture_elevage', 'industrie_agro_chimie',
    'environnement_energie', 'risques_industriels'
]

print("Colonnes dans gdf_factors :", gdf_factors.columns.tolist())

facteurs_valides = [f for f in facteurs if f in gdf_factors.columns]
print("facteurs_valides :", facteurs_valides)

results = []

for target in facteurs_valides:
    features = [f for f in facteurs_valides if f != target]
    print(f"\nModèle pour cible : {target} avec features : {features}")

    X = gdf_factors[features]
    y = gdf_factors[target]

    data = pd.concat([X, y], axis=1).dropna()
    print(f"Données valides (sans NaN) pour {target} :", data.shape)

    if data.empty:
        print(f"Aucune donnée valide pour {target}, on passe à la cible suivante.")
        continue

    X_clean = data[features]
    y_clean = data[target]

    X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, random_state=42)

    # Entraînement des modèles
    lr = LinearRegression().fit(X_train, y_train)
    rf = RandomForestRegressor(random_state=42).fit(X_train, y_train)
    hgb = HistGradientBoostingRegressor(random_state=42).fit(X_train, y_train)

    # Prédictions et scores R²
    r2_lr = r2_score(y_test, lr.predict(X_test))
    r2_rf = r2_score(y_test, rf.predict(X_test))
    r2_hgb = r2_score(y_test, hgb.predict(X_test))

    # Déterminer le meilleur modèle
    scores = {'LinReg': r2_lr, 'RandomForest': r2_rf, 'HistGradientBoosting': r2_hgb}
    best_model_name = max(scores, key=scores.get)
    best_r2 = scores[best_model_name]

    print(f"Meilleur modèle pour {target} : {best_model_name} avec R² = {best_r2:.4f}")

    results.append({
        'Cible': target,
        'R2_LinReg': r2_lr,
        'R2_RandomForest': r2_rf,
        'R2_HistGradientBoosting': r2_hgb,
        'Best_Model': best_model_name,
        'Best_R2': best_r2
    })

    # Affichage de l'importance des variables selon le meilleur modèle
    if best_model_name == 'LinReg':
        # Importance via valeur absolue des coefficients
        coefs = pd.Series(abs(lr.coef_), index=X_train.columns).sort_values()
        coefs.plot(kind='barh', title=f"Importance des facteurs pour {target} ({best_model_name})")
        plt.xlabel("Valeur absolue des coefficients")
        plt.tight_layout()
        plt.show()

    elif best_model_name == 'RandomForest':
        # Permutation importance pour RF
        result = permutation_importance(rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)
        importances = pd.Series(result.importances_mean, index=X_test.columns).sort_values()
        importances.plot(kind='barh', title=f"Importance des facteurs pour {target} ({best_model_name} - permutation)")
        plt.xlabel("Importance moyenne par permutation")
        plt.tight_layout()
        plt.show()

    else:  # HistGradientBoosting
        result = permutation_importance(hgb, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)
        importances = pd.Series(result.importances_mean, index=X_test.columns).sort_values()
        importances.plot(kind='barh', title=f"Importance des facteurs pour {target} ({best_model_name} - permutation)")
        plt.xlabel("Importance moyenne par permutation")
        plt.tight_layout()
        plt.show()

if results:
    results_df = pd.DataFrame(results)
    print("\nRésumé des scores R² et meilleurs modèles :")
    print(results_df)
else:
    print("Aucun résultat à afficher (pas de données valides ou pas de facteurs valides).")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression

# XGBoost et CatBoost si installés
try:
    from xgboost import XGBRegressor
    xgb_installed = True
except ImportError:
    xgb_installed = False

try:
    from catboost import CatBoostRegressor
    catboost_installed = True
except ImportError:
    catboost_installed = False

facteurs = [
    'demographie_niveau_vie', 'emploi_activite', 'acces_sante',
    'services_publics', 'agriculture_elevage', 'industrie_agro_chimie',
    'environnement_energie', 'risques_industriels'
]

facteurs_valides = [f for f in facteurs if f in gdf_factors.columns]
results = []

for target in facteurs_valides:
    features = [f for f in facteurs_valides if f != target]
    print(f"\nCible : {target}")

    X = gdf_factors[features].copy()
    y = gdf_factors[target]

    # 1️⃣ Imputation KNN
    imputer = KNNImputer(n_neighbors=5)
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

    # 2️⃣ Création d'interactions simples et triples
    cols = X_imputed.columns.tolist()
    # Interactions simples
    for i in range(len(cols)):
        for j in range(i+1, len(cols)):
            X_imputed[f'{cols[i]}*{cols[j]}'] = X_imputed[cols[i]] * X_imputed[cols[j]]
    # Interactions triples (si dataset pas trop petit)
    for i in range(len(cols)):
        for j in range(i+1, len(cols)):
            for k in range(j+1, len(cols)):
                X_imputed[f'{cols[i]}*{cols[j]}*{cols[k]}'] = X_imputed[cols[i]] * X_imputed[cols[j]] * X_imputed[cols[k]]

    # 3️⃣ Transformations log1p et sqrt
    for col in X_imputed.columns:
        if (X_imputed[col] > 0).all():
            X_imputed[f'log1p_{col}'] = np.log1p(X_imputed[col])
            X_imputed[f'sqrt_{col}'] = np.sqrt(X_imputed[col])

    # 4️⃣ Standardisation
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_imputed)

    # 5️⃣ Sélection automatique des 10 meilleures features
    k_best = min(10, X_scaled.shape[1])
    selector = SelectKBest(score_func=f_regression, k=k_best)
    X_selected = selector.fit_transform(X_scaled, y)
    selected_features = [X_imputed.columns[i] for i, s in enumerate(selector.get_support()) if s]

    # 6️⃣ Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_selected, y, test_size=0.2, random_state=42
    )

    # 7️⃣ Définition des modèles de base pour stacking
    estimators = [
        ('rf', RandomForestRegressor(random_state=42, n_estimators=200, max_depth=5)),
        ('hgb', HistGradientBoostingRegressor(random_state=42, max_iter=200))
    ]
    if xgb_installed:
        estimators.append(('xgb', XGBRegressor(random_state=42, n_estimators=200, max_depth=5, learning_rate=0.1)))
    if catboost_installed:
        estimators.append(('cat', CatBoostRegressor(random_state=42, iterations=200, verbose=0)))

    # 8️⃣ Stacking
    stack_model = StackingRegressor(
        estimators=estimators,
        final_estimator=LinearRegression(),
        n_jobs=-1
    )
    stack_model.fit(X_train, y_train)

    # 9️⃣ Évaluation
    y_pred = stack_model.predict(X_test)
    test_r2 = r2_score(y_test, y_pred)
    cv_r2 = cross_val_score(stack_model, X_selected, y, cv=5, scoring='r2').mean()
    print(f"Stacking R² test = {test_r2:.4f}, CV R² = {cv_r2:.4f}")

    results.append({
        'Cible': target,
        'R2_Test': test_r2,
        'R2_CV': cv_r2,
        'Best_Model': 'Stacking'
    })

    # 10️⃣ Importance des variables
    perm = permutation_importance(stack_model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)
    importances = pd.Series(perm.importances_mean, index=selected_features).sort_values()
    importances.plot(kind='barh', title=f"Importance pour {target} (Stacking Ultra)")
    plt.xlabel("Importance moyenne par permutation")
    plt.show()

# Résumé final
results_df = pd.DataFrame(results)
print("\nRésumé des R² et meilleurs modèles :")
print(results_df)
